<h2>Demo</h2>
<p>Here are few examples for demonstrating A* algorithm</p>
<h4>1. Finding shortest path in the graphs</h4>
<p>Here we can see the comparison between the Dijkstras algorithm and A* search algorithm. It also shows why we need to devise a proper heuristic function for A*, or else Dijkstras algorithm is more suitable for calculating shortest path. The below demonstration is interactive where you can select the number of nodes in the graph using a slider. You can also choose the start node and end node to calculate the shortest path.</p>
<p>The graph used here is watts_strogatz_graph (a random graph generator) with nearest neighbours as $8$ and probability of the connectivity between nodes is $0.2$. </p>
<p>Assuming weight of all edges is $1$</p>
<p> Heuristic function used here is L1 norm between current node and end node: $L1 = ||current\_node - end\_node||$</p>

In [4]:
import networkx as nx
import time
import ipywidgets as widgets
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt

seed = 6020

In [11]:
canvas = []
nodes_slider = widgets.IntSlider(description="nodes: ",max=5000, min=500, step=200, value=1000)
start_node = widgets.IntText(description="Start node: ", value=0)
end_node = widgets.IntText(description="End node: ", value=0)
canvas+=[nodes_slider, start_node, end_node]

output = widgets.Output()

def slider_update(change):
    
    nodes=nodes_slider.get_interact_value()
    start=start_node.get_interact_value() 
    end=end_node.get_interact_value()
    with output:
        output.clear_output(wait=True)
        G = nx.watts_strogatz_graph(nodes,8,0.2,seed)
        start_time = time.process_time_ns()
        dij_path = nx.dijkstra_path(G,start,end)
        end_time = time.process_time_ns()
        print("Time taken to compute Dijkstra's path: ",end_time-start_time)
        dij_graph = nx.Graph([(dij_path[i],dij_path[i+1]) for i in range(len(dij_path)-1)])
        plt.figure(figsize=(6, 4))
        nx.draw(
            dij_graph,
            with_labels=True,
            node_color="lightgreen",
            edge_color="blue",
            node_size=600,
            font_size=10,
        )
        plt.show()
        start_time = time.process_time_ns()
        astar_path = nx.astar_path(G,start,end,lambda a,b: abs(b-a))
        end_time = time.process_time_ns()
        print("Time taken to compute Astar path: ", end_time-start_time)
        astar_graph = nx.Graph(nx.Graph([(astar_path[i],astar_path[i+1]) for i in range(len(astar_path)-1)]))
        plt.figure(figsize=(6, 4))
        nx.draw(
            astar_graph,
            with_labels=True,
            node_color="lightgreen",
            edge_color="blue",
            node_size=600,
            font_size=10,
        )
        plt.show()

nodes_slider.observe(slider_update, names="value")
start_node.observe(slider_update, names="value")
end_node.observe(slider_update, names="value")

display(widgets.VBox(canvas), output)

Output()

<h4>Observation</h4>
<p>As you can see the computation time differs for both algorithm when number of nodes is $3700$, start node is $0$ and end node is $100$. For Dijkstras it took $15625000\space ns$ (nano seconds), and for A* it took $0\space ns$. But the caveat is A* doesn't give you the shortest path which is of length $8$ whereas Dijkstras gives you the optimal path of length $5$. Therefore we can conclude that if we don't get the heuristics correct, A* will not work correctly as we imagined.</p>

<h4>2. A* algorithm used in grids</h4>
<p>We will create a grid of size $n\times n$, where you can choose the grid size $n$, start node and end node to calculate shortest path. You can also add blocks in the grid. After placing blocks (if any), start and end nodes, just click <b>calculate</b> button to find the shortest path using A*. After the calculation, you can see green-colored cells in the grid which denotes the shortest path between start and end node</p>

In [6]:
canvas = []
grid = None
grid_size = None
grid_graph = None
cells = None
block = None
start = None
end = None
check_types = {}

inp_grid_size = widgets.IntText(description='size', value=5)
output = widgets.Output()

canvas+=[inp_grid_size]

def heuristics(a,b):
    x0,y0 = int(a[0]), int(a[1])
    x1,y1 = int(b[0]), int(b[1])
    return ((x1-x0)**2 + (y1-y0)**2)**0.5

def toggler(change):
    owner_name = change.get("owner",{"description": ""}).description
    for key in check_types.keys():
        if key != owner_name:
            check_types[key]["obj"].value = False

def update_grid(widget):
    global start_node, end_node
    if (widget.description == "Calculate"):
        start_time = time.process_time_ns()
        astar_paths = nx.astar_path(grid_graph, start_node, end_node, heuristics)
        end_time = time.process_time_ns()
        with output:
            print("Time to calculate the path: ", end_time - start_time)
        # Highlight paths of astar
        for node in astar_paths[1:-1]:
            i,j = node[0], node[1]
            cells[i].children[j].style.button_color="green"
             
    else:
        # for cells
        x,y = [int(i) for i in widget.description.split(",")]
        cell_type = [val for val in check_types.values() if val["obj"].value]
        if cell_type:
            cell = cell_type[0]
            if cell["ops"] == "remove_cell":
                if cell["color"] == widget.style.button_color:
                    edges = [((x-i,y-j),(x,y)) for (i,j) in [(0,1),(1,0), (0,-1), (-1,0), (1,1),(1,-1), (-1,-1), (-1,1)] if (0<=(x-i)<grid_size) and (0<=(y-j)<grid_size)]
                    grid_graph.add_edges_from(edges)
                    widget.style.button_color = "lightgrey"
                    return
                grid_graph.remove_node((x,y))
            elif cell["ops"] == "set_start_node":
                start_node = (x,y)
            elif cell["ops"] == "set_end_node":
                end_node = (x,y)
            widget.style.button_color = cell["color"]
            return
        widget.style.button_color = "lightgrey"


def reset(_):
    create_grid()

def create_grid(change=None):
    global grid_graph, cells, grid_size
    grid_size = inp_grid_size.get_interact_value()
    grid_graph = nx.grid_graph((grid_size,grid_size))
    for node in grid_graph.nodes():
        x,y = node
        edges = [((x-i,y-j),(x,y)) for (i,j) in [(1,1),(1,-1), (-1,-1), (-1,1)] if (0<=(x-i)<grid_size) and (0<=(y-j)<grid_size)]
        grid_graph.add_edges_from(edges)
    with output:
        output.clear_output(wait=True)
        plot_canvas = []
        cells = []
        for i in range(grid_size):
            row = []
            for j in range(grid_size):
                cell = widgets.Button(
                    description=f"{i},{j}",  
                    layout=widgets.Layout(width='20px', height='20px'),
                    style=widgets.ButtonStyle(font_size="10px", button_color="lightgrey")
                    ) 
                cell.on_click(update_grid)
                row.append(cell)
            cells.append(widgets.HBox(row))
        plot_canvas.append(widgets.VBox(cells))
        
        # Blocks
        block = widgets.Checkbox(description="Add Block")
        block.observe(toggler)
        check_types["Add Block"] = {"obj": block, "color": "black", "ops": "remove_cell"}
        # start node
        start_node_inp = widgets.Checkbox(description="Start Node")
        start_node_inp.observe(toggler)
        check_types["Start Node"] = {"obj": start_node_inp, "color": "blue", "ops": "set_start_node"}
        # end node
        end_node_inp = widgets.Checkbox(description="End Node")
        end_node_inp.observe(toggler)
        check_types["End Node"] = {"obj": end_node_inp, "color": "red", "ops": "set_end_node"}
        # Calculate distance
        calc_btn = widgets.Button(description="Calculate")
        calc_btn.on_click(update_grid)
        # Reset entire grid
        reset_btn = widgets.Button(description="Reset")
        reset_btn.on_click(reset)
        # Add it to canvas plot
        plot_canvas .append(widgets.VBox([block, start_node_inp, end_node_inp, calc_btn,reset_btn]))
        display(widgets.HBox(plot_canvas))          

inp_grid_size.observe(create_grid, names="value")
display(widgets.HBox(canvas), output)
create_grid()

Output()

<h4>Observation</h4>
<p>For below state of the grid:</p>
<ol>
    <li>Grid size: $5 \times 5$</li>
    <li>Blocks are placed at: $(2,1), (2,2), (1,2)$</li>
    <li>Start Node: $(0,0)$</li>
    <li>End Node: $(3,3)$</li>
</ol>
<p>The reason for selecting $(1,1)$ as the first cell in the shortest path is because the value of heuristic function evaluated at $(1,1)$ is lesser compared to other cells $(1,0), (0,1)$. </p>
<center>$Heuristics((1,1),(3,3)) = 2.83$</center>
<center>$Heuristics((1,0),(3,3)) = 3.61$</center>
<center>$Heuristics((0,1),(3,3)) = 3.61$</center>

In [ ]:
# Sample code from https://www.redblobgames.com/pathfinding/a-star/
# Copyright 2014 Red Blob Games <redblobgames@gmail.com>
#
# Feel free to use this code in your own projects, including commercial projects
# License: Apache v2.0 <http://www.apache.org/licenses/LICENSE-2.0.html>

from __future__ import annotations
# some of these types are deprecated: https://www.python.org/dev/peps/pep-0585/
from typing import Protocol, Iterator, Tuple, TypeVar, Optional
T = TypeVar('T')

Location = TypeVar('Location')
class Graph(Protocol):
    def neighbors(self, id: Location) -> list[Location]: pass

class SimpleGraph:
    def __init__(self):
        self.edges: dict[Location, list[Location]] = {}
    
    def neighbors(self, id: Location) -> list[Location]:
        return self.edges[id]

example_graph = SimpleGraph()
example_graph.edges = {
    'A': ['B'],
    'B': ['C'],
    'C': ['B', 'D', 'F'],
    'D': ['C', 'E'],
    'E': ['F'],
    'F': [],
}

import collections

class Queue:
    def __init__(self):
        self.elements = collections.deque()
    
    def empty(self) -> bool:
        return not self.elements
    
    def put(self, x: T):
        self.elements.append(x)
    
    def get(self) -> T:
        return self.elements.popleft()

# utility functions for dealing with square grids
def from_id_width(id, width):
    return (id % width, id // width)

def draw_tile(graph, id, style):
    r = " . "
    if 'number' in style and id in style['number']: r = " %-2d" % style['number'][id]
    if 'point_to' in style and style['point_to'].get(id, None) is not None:
        (x1, y1) = id
        (x2, y2) = style['point_to'][id]
        if x2 == x1 + 1: r = " > "
        if x2 == x1 - 1: r = " < "
        if y2 == y1 + 1: r = " v "
        if y2 == y1 - 1: r = " ^ "
    if 'path' in style and id in style['path']:   r = " @ "
    if 'start' in style and id == style['start']: r = " A "
    if 'goal' in style and id == style['goal']:   r = " Z "
    if id in graph.walls: r = "###"
    return r

def draw_grid(graph, **style):
    print("___" * graph.width)
    for y in range(graph.height):
        for x in range(graph.width):
            print("%s" % draw_tile(graph, (x, y), style), end="")
        print()
    print("~~~" * graph.width)

# data from main article
DIAGRAM1_WALLS = [from_id_width(id, width=30) for id in [21,22,51,52,81,82,93,94,111,112,123,124,133,134,141,142,153,154,163,164,171,172,173,174,175,183,184,193,194,201,202,203,204,205,213,214,223,224,243,244,253,254,273,274,283,284,303,304,313,314,333,334,343,344,373,374,403,404,433,434]]

GridLocation = Tuple[int, int]

class SquareGrid:
    def __init__(self, width: int, height: int):
        self.width = width
        self.height = height
        self.walls: list[GridLocation] = []
    
    def in_bounds(self, id: GridLocation) -> bool:
        (x, y) = id
        return 0 <= x < self.width and 0 <= y < self.height
    
    def passable(self, id: GridLocation) -> bool:
        return id not in self.walls
    
    def neighbors(self, id: GridLocation) -> Iterator[GridLocation]:
        (x, y) = id
        neighbors = [(x+1, y), (x-1, y), (x, y-1), (x, y+1)] # E W N S
        # see "Ugly paths" section for an explanation:
        if (x + y) % 2 == 0: neighbors.reverse() # S N W E
        results = filter(self.in_bounds, neighbors)
        results = filter(self.passable, results)
        return results

class WeightedGraph(Graph):
    def cost(self, from_id: Location, to_id: Location) -> float: pass

class GridWithWeights(SquareGrid):
    def __init__(self, width: int, height: int):
        super().__init__(width, height)
        self.weights: dict[GridLocation, float] = {}
    
    def cost(self, from_node: GridLocation, to_node: GridLocation) -> float:
        return self.weights.get(to_node, 1)

diagram4 = GridWithWeights(10, 10)
diagram4.walls = [(1, 7), (1, 8), (2, 7), (2, 8), (3, 7), (3, 8)]
diagram4.weights = {loc: 5 for loc in [(3, 4), (3, 5), (4, 1), (4, 2),
                                       (4, 3), (4, 4), (4, 5), (4, 6),
                                       (4, 7), (4, 8), (5, 1), (5, 2),
                                       (5, 3), (5, 4), (5, 5), (5, 6),
                                       (5, 7), (5, 8), (6, 2), (6, 3),
                                       (6, 4), (6, 5), (6, 6), (6, 7),
                                       (7, 3), (7, 4), (7, 5)]}

import heapq

class PriorityQueue:
    def __init__(self):
        self.elements: list[tuple[float, T]] = []
    
    def empty(self) -> bool:
        return not self.elements
    
    def put(self, item: T, priority: float):
        heapq.heappush(self.elements, (priority, item))
    
    def get(self) -> T:
        return heapq.heappop(self.elements)[1]

def dijkstra_search(graph: WeightedGraph, start: Location, goal: Location):
    frontier = PriorityQueue()
    frontier.put(start, 0)
    came_from: dict[Location, Optional[Location]] = {}
    cost_so_far: dict[Location, float] = {}
    came_from[start] = None
    cost_so_far[start] = 0
    
    while not frontier.empty():
        current: Location = frontier.get()
        
        if current == goal:
            break
        
        for next in graph.neighbors(current):
            new_cost = cost_so_far[current] + graph.cost(current, next)
            if next not in cost_so_far or new_cost < cost_so_far[next]:
                cost_so_far[next] = new_cost
                priority = new_cost
                frontier.put(next, priority)
                came_from[next] = current
    
    return came_from, cost_so_far

# thanks to @m1sp <Jaiden Mispy> for this simpler version of
# reconstruct_path that doesn't have duplicate entries

def reconstruct_path(came_from: dict[Location, Location],
                     start: Location, goal: Location) -> list[Location]:

    current: Location = goal
    path: list[Location] = []
    if goal not in came_from: # no path was found
        return []
    while current != start:
        path.append(current)
        current = came_from[current]
    path.append(start) # optional
    path.reverse() # optional
    return path

diagram_nopath = GridWithWeights(10, 10)
diagram_nopath.walls = [(5, row) for row in range(10)]

def heuristic(a: GridLocation, b: GridLocation) -> float:
    (x1, y1) = a
    (x2, y2) = b
    return abs(x1 - x2) + abs(y1 - y2)

def a_star_search(graph: WeightedGraph, start: Location, goal: Location):
    frontier = PriorityQueue()
    frontier.put(start, 0)
    came_from: dict[Location, Optional[Location]] = {}
    cost_so_far: dict[Location, float] = {}
    came_from[start] = None
    cost_so_far[start] = 0
    
    while not frontier.empty():
        current: Location = frontier.get()
        
        if current == goal:
            break
        
        for next in graph.neighbors(current):
            new_cost = cost_so_far[current] + graph.cost(current, next)
            if next not in cost_so_far or new_cost < cost_so_far[next]:
                cost_so_far[next] = new_cost
                priority = new_cost + heuristic(next, goal)
                frontier.put(next, priority)
                came_from[next] = current
    
    return came_from, cost_so_far

def breadth_first_search(graph: Graph, start: Location, goal: Location):
    frontier = Queue()
    frontier.put(start)
    came_from: dict[Location, Optional[Location]] = {}
    came_from[start] = None
    
    while not frontier.empty():
        current: Location = frontier.get()
        
        if current == goal:
            break
        
        for next in graph.neighbors(current):
            if next not in came_from:
                frontier.put(next)
                came_from[next] = current
    
    return came_from

class SquareGridNeighborOrder(SquareGrid):
    def neighbors(self, id):
        (x, y) = id
        neighbors = [(x + dx, y + dy) for (dx, dy) in self.NEIGHBOR_ORDER]
        results = filter(self.in_bounds, neighbors)
        results = filter(self.passable, results)
        return list(results)

def test_with_custom_order(neighbor_order):
    if neighbor_order:
        g = SquareGridNeighborOrder(30, 15)
        g.NEIGHBOR_ORDER = neighbor_order
    else:
        g = SquareGrid(30, 15)
    g.walls = DIAGRAM1_WALLS
    start, goal = (8, 7), (27, 2)
    came_from = breadth_first_search(g, start, goal)
    draw_grid(g, path=reconstruct_path(came_from, start=start, goal=goal),
              point_to=came_from, start=start, goal=goal)

class GridWithAdjustedWeights(GridWithWeights):
    def cost(self, from_node, to_node):
        prev_cost = super().cost(from_node, to_node)
        nudge = 0
        (x1, y1) = from_node
        (x2, y2) = to_node
        if (x1 + y1) % 2 == 0 and x2 != x1: nudge = 1
        if (x1 + y1) % 2 == 1 and y2 != y1: nudge = 1
        return prev_cost + 0.001 * nudge